In [8]:
import tensorflow_hub as hub

modelPath = 'models/lightning-v4'
model = hub.load("https://tfhub.dev/google/movenet/singlepose/lightning/4")
movenet = model.signatures['serving_default']




In [10]:
# dir(model)
model.summary()

AttributeError: '_UserObject' object has no attribute 'summary'

In [6]:
tf.__version__

'2.10.0'